In [1]:
from agential.cog.agent.reflexion import ReflexionCoTAgent
from agential.cog.prompts.agent.reflexion import (
    REFLEXION_COT_FEWSHOT_EXAMPLES,
    REFLEXION_COT_INSTRUCTION,
    REFLEXION_COT_REFLECT_FEWSHOT_EXAMPLES,
    REFLEXION_COT_REFLECT_INSTRUCTION
)

from langchain_openai import ChatOpenAI

import dotenv
dotenv.load_dotenv()

import warnings
warnings.filterwarnings('ignore')

llm = ChatOpenAI()

agent = ReflexionCoTAgent(
    llm=llm,
    mode={"qa": "hotpotqa"}
)

In [2]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"
context = 'VIVA Media GmbH (until 2004 "VIVA Media AG") is a music television network originating from Germany. It was founded for broadcast of VIVA Germany as VIVA Media AG in 1993 and has been owned by their original concurrent Viacom, the parent company of MTV, since 2004. Viva channels exist in some European countries; the first spin-offs were launched in Poland and Switzerland in 2000.\n\nA Gesellschaft mit beschränkter Haftung (] , abbreviated GmbH ] and also GesmbH in Austria) is a type of legal entity very common in Germany, Austria, Switzerland (where it is equivalent to a S.à r.l.) and Liechtenstein. In the United States, the equivalent type of entity is the limited liability company (LLC). The name of the GmbH form emphasizes the fact that the owners ("Gesellschafter", also known as members) of the entity are not personally liable for the company\'s debts. "GmbH"s are considered legal persons under German and Austrian law. Other variations include mbH (used when the term "Gesellschaft" is part of the company name itself), and gGmbH ("gemeinnützige" GmbH) for non-profit companies.'

out = agent.generate(
    question=question, 
    key=key, 
    examples=REFLEXION_COT_FEWSHOT_EXAMPLES,
    reflection_strategy="last_attempt_and_reflexion",
    prompt=REFLEXION_COT_INSTRUCTION,
    reflect_examples=REFLEXION_COT_REFLECT_FEWSHOT_EXAMPLES,
    reflect_prompt=REFLEXION_COT_REFLECT_INSTRUCTION,
    additional_keys={},
    reflection_additional_keys={},
    max_trials=3,
    patience=3,
    reset=True
)

<PROMPT AGENT=====================================================>
Solve a question answering task by having a Thought, then Finish with your answer. Thought can reason about the current situation. Finish[answer] returns the answer and finishes the task.
Here are some examples:
Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: Let's think step by step. The eastern sector of Colorado orogeny extends into the High Plains. High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]

Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Thought: Let's think step by step. Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.
Action: Finish[Richard Nixon]

Question: Which documentary is about Finnish rock groups, Adam Clayton Powell or

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


<OUT AGENT=====================================================>
'Let\'s think step by step. VIVA Media AG changed its name to VIVA Entertainment in 2004. The acronym VIVA stands for "Videoverwertungsanstalt." \nAction: Finish[Videoverwertungsanstalt]'
<OUT AGENT=====================================================>
<PROMPT AGENT=====================================================>
Solve a question answering task by having a Thought, then Finish with your answer. Thought can reason about the current situation. Finish[answer] returns the answer and finishes the task.
Here are some examples:
Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: Let's think step by step. The eastern sector of Colorado orogeny extends into the High Plains. High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]

Question: Musician and satirist Allie Goertz wrote a 

In [3]:
out

[{'thought': 'Let\'s think step by step. VIVA Media AG changed its name to VIVA Entertainment in 2004. The acronym VIVA stands for "Videoverwertungsanstalt." ',
  'action_type': 'Finish',
  'query': 'Videoverwertungsanstalt',
  'obs': 'Answer is INCORRECT',
  'answer': 'Videoverwertungsanstalt',
  'is_correct': False},
 {'thought': 'I need to research the specific details of the company name change thoroughly to provide an accurate answer. This will help me avoid making inaccurate assumptions or guesses.',
  'action_type': '',
  'query': '',
  'obs': 'Invalid action type, please try again.',
  'answer': 'Videoverwertungsanstalt',
  'is_correct': False},
 {'thought': 'I need to research the history and rebranding of VIVA Media AG in 2004 to understand the reason behind the name change and the new acronym. This will help me provide an accurate answer. ',
  'action_type': '',
  'query': '',
  'obs': 'Invalid action type, please try again.',
  'answer': 'Videoverwertungsanstalt',
  'is_cor